In [5]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import skimage.io
import skimage.color as color
import skimage.segmentation
import skimage.measure
from scipy.stats import skew

# Parameter
images_folder = "fluo_corrected"  # Ordner mit TIFF-Bildern
masks_folder = "masks"            # Ordner mit PNG-Masken
output_folder = "histogram_data"  # Ordner zum Speichern der txt-Dateien
os.makedirs(output_folder, exist_ok=True)

# Liste aller TIFF-Dateien im Bilderordner
image_files = [f for f in os.listdir(images_folder) if f.lower().endswith('.tif')]

if not image_files:
    print("Keine TIFF-Dateien im Ordner 'images' gefunden.")
    sys.exit(1)


for image_file in image_files:
    # Entferne das 'corrected_' Präfix falls vorhanden
    base_name = image_file
    if base_name.startswith("corrected_"):
        base_name = base_name[len("corrected_"):]

    base_name = os.path.splitext(base_name)[0]
    mask_file = base_name + "_cp_masks.tif"
    mask_path = os.path.join(masks_folder, mask_file)
    txt_output_path = os.path.join(output_folder, base_name + "_median_intensities.txt")

    image_path = os.path.join(images_folder, image_file)

    # Prüfen, ob die Maske existiert
    if not os.path.isfile(mask_path):
        print(f"Warnung: Maske {mask_path} nicht gefunden für Bild {image_file}. Überspringe.")
        continue

    # Bild und Maske laden
    image = skimage.io.imread(image_path)
    mask = skimage.io.imread(mask_path)

    # Konvertiere Bild ggf. in Graustufen
    intensity_image = image

    # Eigenschaften der Regionen berechnen
    regions = skimage.measure.regionprops(mask, intensity_image=intensity_image)
    print(len(regions))
    # extract median intensity 
    valid_intensities_median = [np.median(intensity_image[region.coords[:,0], region.coords[:,1]])
                                for region in regions]

    # Extrahiere mean Intensitäten
    valid_intensities = [np.mean(intensity_image[region.coords[:, 0], region.coords[:, 1]]) 
                         for region in regions if region.intensity_image is not None]
    # skewness of distribution
    skewness_values = []

    for region in regions:
        intensities = intensity_image[region.coords[:, 0], region.coords[:, 1]]
        skew_val = skew(intensities, bias=False)  # `bias=False` for better statistical accuracy
        skewness_values.append(skew_val)

    if not valid_intensities:
        print(f"Keine gültigen Intensitäten für {image_file}. Überspringe.")
        continue
    # Speichern
    mean_output_path = os.path.join(output_folder, base_name + "_mean_intensities.txt") 
    median_output_path = os.path.join(output_folder, base_name + "_median_intensities.txt")
    skew_output_path = os.path.join(output_folder, base_name + "_skewness.txt")
    
    with open(mean_output_path, "w") as f:
        for intensity in valid_intensities:
            f.write(f"{intensity}\n")
    with open(median_output_path,"w") as f:
        for intensity in valid_intensities_median:
            f.write(f"{intensity}\n")
    with open(skew_output_path, "w") as f:
        for val in skewness_values:
            f.write(f"{val}\n")

1067
211
311
170
104
185
175
1007
953
811
330
587
780
251
478
303
